In [129]:
# RoadMap
# 1. Import our dependencies
# 2. Read our datsets
# 3. Split train and testing
# 4. declare necessary hyperparametres
# 5. declare placeholders
# 6. declare variables (weights and biases)
# 7. build function for conv2d and max-pooling for usability
# 8. Build function for conv net 
# 9. define cost and optimizer 
# 10. train and test

# 1. Import our necessary dependencies

In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import scipy.misc as smp
%matplotlib inline

# 2. Import our dataset

In [131]:
df = pd.read_pickle('/data_to_upload/data_batch_1')
X = df['data']
y = df['labels']
X

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ..., 
       [ 71,  60,  74, ...,  68,  69,  68],
       [250, 254, 211, ..., 215, 255, 254],
       [ 62,  61,  60, ..., 130, 130, 131]], dtype=uint8)

In [132]:
print("After importing data: ", X.shape)
X = np.array(X, dtype=float) / 255.0
print("After converting array: ", X.shape)
X = np.reshape(X, [-1, 3 , 32 ,32])
print("After reshape: ", X.shape)
X = np.transpose(X, [0, 2, 3, 1])
print("After transpose: ", X.shape)
X = np.reshape(X, [-1, 32*32*3])
print("After tra reshape: ", X.shape)

After importing data:  (10000, 3072)
After converting array:  (10000, 3072)
After reshape:  (10000, 3, 32, 32)
After transpose:  (10000, 32, 32, 3)
After tra reshape:  (10000, 3072)


In [133]:
X

array([[ 0.23137255,  0.24313725,  0.24705882, ...,  0.48235294,
         0.36078431,  0.28235294],
       [ 0.60392157,  0.69411765,  0.73333333, ...,  0.56078431,
         0.52156863,  0.56470588],
       [ 1.        ,  1.        ,  1.        , ...,  0.31372549,
         0.3372549 ,  0.32941176],
       ..., 
       [ 0.27843137,  0.30196078,  0.17254902, ...,  0.32941176,
         0.30196078,  0.26666667],
       [ 0.98039216,  1.        ,  1.        , ...,  1.        ,
         0.99607843,  0.99607843],
       [ 0.24313725,  0.21568627,  0.02745098, ...,  0.65490196,
         0.77647059,  0.51372549]])

In [134]:
y = np.array(y, dtype=float)
y.shape

(10000,)

In [135]:
y = y.reshape(-1, 1)
y.shape

(10000, 1)

In [136]:
oneHotEncode = OneHotEncoder()
oneHotEncode.fit(y)
y = oneHotEncode.transform(y).toarray()
y.shape

(10000, 10)

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [138]:
X_train.shape

(7000, 3072)

In [139]:
# single_img_reshaped = np.transpose(np.reshape(X,(3, 32,32)), (1,2,0))
# single_img_reshaped.shape

In [140]:
# hyperparametres
learning_rate = 0.003
num_steps = 100
batch_size = 128
display_step = 10

dropout = 0.75

X = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

weights = {
    'wh1' : tf.Variable(tf.random_normal([5, 5, 3, 64])),
    'wh2' : tf.Variable(tf.random_normal([5, 5, 64, 128])),
    'wh3' : tf.Variable(tf.random_normal([5, 5 ,128, 256])),
    'wh4' : tf.Variable(tf.random_normal([5, 5, 256, 512])),
    'wf1' : tf.Variable(tf.random_normal([2*2*512, 1024])),
    'wf2' : tf.Variable(tf.random_normal([1024, 2048])),
    'wOut' : tf.Variable(tf.random_normal([2048, 10]))
}

biases = {
    'bh1' : tf.Variable(tf.random_normal([64])),
    'bh2' : tf.Variable(tf.random_normal([128])),
    'bh3' : tf.Variable(tf.random_normal([256])),
    'bh4' : tf.Variable(tf.random_normal([512])),
    'bf1' : tf.Variable(tf.random_normal([1024])),
    'bf2' : tf.Variable(tf.random_normal([2048])),
    'bOut' : tf.Variable(tf.random_normal([10]))
}

In [141]:
def conv_2d(X, weights, biases, strides=1):
    X = tf.nn.conv2d(X, weights, strides=[1, strides, strides, 1], padding="SAME")
    X = tf.nn.bias_add(X, biases)
    return tf.nn.relu(X)
def max_pooling(X, k=2):
    return tf.nn.max_pool(X, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

def conv_net(X, weights, biases, dropout):
    X = tf.reshape(X, shape=[-1, 32, 32, 3])
    
    conv1 = conv_2d(X, weights=weights['wh1'], biases=biases['bh1'])
    conv1 = max_pooling(conv1)
    conv1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    conv2 = conv_2d(conv1, weights=weights['wh2'], biases=biases['bh2'])
    conv2 = max_pooling(conv2)
    conv2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    conv3 = conv_2d(conv2, weights=weights['wh3'], biases=biases['bh3'])
    conv3 = max_pooling(conv3)
    conv3 = tf.nn.lrn(conv3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    conv4 = conv_2d(conv3, weights=weights['wh4'], biases=biases['bh4'])
    conv4 = max_pooling(conv4)
    conv4 = tf.nn.lrn(conv4, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    fc1 = tf.reshape(conv4, [-1, weights['wf1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wf1']), biases['bf1'])
    fc1 = tf.nn.relu(fc1)
    
    fc2 = tf.add(tf.matmul(fc1, weights['wf2']), biases['bf2'])
    fc2 = tf.nn.relu(fc2)
    
    fc2 = tf.nn.dropout(fc2, dropout)
    
    out = tf.add(tf.matmul(fc2, weights['wOut']), biases['bOut'])
    return out

In [142]:
logits = conv_net(X, weights=weights, biases=biases, dropout=keep_prob)
prediction = tf.nn.softmax(logits=logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [143]:

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        
        randidx = np.random.randint(len(X_train), size=batch_size)
        batch_xs = X_train[randidx]
        batch_ys = y_train[randidx]
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_xs,
                                                                 y: batch_ys,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      y: y_test,
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 401777.0938, Training Accuracy= 0.117
Step 10, Minibatch Loss= 208893.1875, Training Accuracy= 0.148
Step 20, Minibatch Loss= 90398.6094, Training Accuracy= 0.195


KeyboardInterrupt: 

In [ ]:
Step 1, Minibatch Loss= 147477.0625, Training Accuracy= 0.164
Step 10, Minibatch Loss= 45248.9023, Training Accuracy= 0.227
Step 20, Minibatch Loss= 21720.9219, Training Accuracy= 0.227
Step 30, Minibatch Loss= 10624.4209, Training Accuracy= 0.289
Step 40, Minibatch Loss= 6556.7979, Training Accuracy= 0.211
Step 50, Minibatch Loss= 3436.7925, Training Accuracy= 0.344
Step 60, Minibatch Loss= 1435.6462, Training Accuracy= 0.289
Step 70, Minibatch Loss= 335.0975, Training Accuracy= 0.117
Step 80, Minibatch Loss= 105.2235, Training Accuracy= 0.148
Step 90, Minibatch Loss= 23.6756, Training Accuracy= 0.062
Step 100, Minibatch Loss= 48.1269, Training Accuracy= 0.125
Optimization Finished!
Testing Accuracy: 0.0916667